In [31]:
import numpy as np
import duckdb
import pandas as pd
import seaborn as sns

In [3]:
data = pd.read_csv("BDC_2024_Womens_Data.csv")

In [5]:
data.columns

Index(['Date', 'Home Team', 'Away Team', 'Period', 'Clock',
       'Home Team Skaters', 'Away Team Skaters', 'Home Team Goals',
       'Away Team Goals', 'Team', 'Player', 'Event', 'X Coordinate',
       'Y Coordinate', 'Detail 1', 'Detail 2', 'Detail 3', 'Detail 4',
       'Player 2', 'X Coordinate 2', 'Y Coordinate 2'],
      dtype='object')

In [10]:
shot_attempts = duckdb.sql("""
                            SELECT Player, COUNT(Event)
                            FROM data
                            WHERE Event = 'Goal' OR Event = 'Shot'
                            GROUP BY Player""")

In [11]:
shot_attempts

┌─────────────────────┬────────────────┐
│       Player        │ count("Event") │
│       varchar       │     int64      │
├─────────────────────┼────────────────┤
│ Hilary Knight       │             16 │
│ Brianne Jenner      │             21 │
│ Taylor Heise        │             16 │
│ Alex Carpenter      │             15 │
│ Megan Keller        │             14 │
│ Grace Zumwinkle     │              3 │
│ Marie-Philip Poulin │             16 │
│ Renata Fast         │             20 │
│ Jocelyne Larocque   │             16 │
│ Hannah Bilka        │             12 │
│     ·               │              · │
│     ·               │              · │
│     ·               │              · │
│ Lacey Eden          │              7 │
│ Nicole Gosling      │              4 │
│ Kirsten Simms       │              7 │
│ Caroline Harvey     │              5 │
│ Ashton Bell         │              3 │
│ Jennifer Gardiner   │              2 │
│ Loren Gabel         │              5 │
│ Anna Wilgren  

In [29]:
# Another possible direction? -- What differentiates a goal vs a shot?
# Who are the good goalies of the game?

# On net but unsuccessful = goalie saved
# One-timer: a shot that occurs when a player meets a teammate's pass with an 
# immediate slapshot, without any attempt to control the puck on their stick.

# Traffic: a condensed area on the ice that has a high concentration of players 

unsuccessful_shot = duckdb.sql("""
                                SELECT Player, Event,
                               "Detail 1", "Detail 2", "Detail 3", "Detail 4"
                               FROM data
                               WHERE Event = 'Shot'
                               """).df()


unsuccessful_shot

successful_shot = duckdb.sql("""
                                SELECT Player, Event,
                               "Detail 1", "Detail 2", "Detail 3", "Detail 4"
                               FROM data
                               WHERE Event = 'Goal'
                               """).df()


successful_shot

,Player,Event,Detail 1,Detail 2,Detail 3,Detail 4
0,Hilary Knight,Goal,Wristshot,On Net,f,f
1,Brianne Jenner,Goal,Wristshot,On Net,f,f
2,Taylor Heise,Goal,Snapshot,On Net,f,t
3,Alex Carpenter,Goal,Wristshot,On Net,f,f
4,Alex Carpenter,Goal,Snapshot,On Net,f,f
5,Laura Stacey,Goal,Wristshot,On Net,f,f
6,Britta Curl,Goal,Wristshot,On Net,f,f
7,Hannah Bilka,Goal,Snapshot,On Net,f,f
8,Jamie Lee Rattray,Goal,Snapshot,On Net,f,f
9,Gabbie Hughes,Goal,Deflection,On Net,f,f


In [38]:

all_shots = duckdb.sql("""
                        SELECT *
                       FROM unsuccessful_shot
                       FULL JOIN successful_shot
                       ON unsuccessful_shot.Player = successful_shot.Player""")

duckdb.sql("""ALTER TABLE all_shots
           ADD Total Shots
           """)

CatalogException: Catalog Error: Table with name all_shots does not exist!
Did you mean "temp.information_schema.tables"?